In [1]:
model_checkpoint = "bert-base-chinese"
# model_checkpoint = "schen/longformer-chinese-base-4096"
batch_size = 2 # 每一批次的数量
num_labels = 2 # 多少分类，这里是二分类问题，积极和消极
output_dir = "/home/chenli/pre_model/BERT_and_Longformer/BERT" # 模型保存路径
learning_rate = 1e-5 # 学习率
# weight_decay=0.01 # 学习率衰减，设置0.01即可。如果weight_decay设置太小，几乎就不起作用了。
num_train_epochs = 10 # 训练轮次，差不多设置为5左右。轮数不要设置太大。轮数设置的太大，Loss是下降了，但是微调的时候效果不是很好，有可能训练过头了

In [2]:
from datasets import load_dataset
from datasets import load_from_disk
# 加载一个评估标准，默认的评估标准
from datasets import load_metric

In [3]:
# 加载数据集
train_dataset = load_dataset('csv',data_files='../data/MyDataset/travel_comment_data/train_dataset.csv',split='train')
valid_dataset = load_dataset('csv',data_files='../data/MyDataset/travel_comment_data/valid_dataset.csv',split='train')
test_dataset = load_dataset('csv',data_files='../data/MyDataset/travel_comment_data/test_dataset.csv',split='train')

Using custom data configuration default-be87a0d28564f2ae
Reusing dataset csv (/home/chenli/.cache/huggingface/datasets/csv/default-be87a0d28564f2ae/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)
Using custom data configuration default-e6970b6e6097a9c9
Reusing dataset csv (/home/chenli/.cache/huggingface/datasets/csv/default-e6970b6e6097a9c9/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)
Using custom data configuration default-8cb27c720d5c6049
Reusing dataset csv (/home/chenli/.cache/huggingface/datasets/csv/default-8cb27c720d5c6049/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


In [3]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 33858
})

In [4]:
valid_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 4232
})

In [5]:
test_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 4233
})

In [4]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [5]:
# 分词
def preprocess_function(data):
    return tokenizer(data['text'],padding='max_length', max_length=100,truncation=True)

In [6]:
train_encoded_dataset = train_dataset.map(function=preprocess_function,
                     batched=True,
                     remove_columns=['text'])
val_encoded_dataset = valid_dataset.map(function=preprocess_function,
                     batched=True,
                     remove_columns=['text'])
test_encoded_dataset = test_dataset.map(function=preprocess_function,
                     batched=True,
                     remove_columns=['text'])

  0%|          | 0/34 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [11]:
train_encoded_dataset

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 33858
})

In [12]:
val_encoded_dataset

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 4232
})

In [13]:
test_encoded_dataset

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 4233
})

In [7]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [8]:
metric_name = "accuracy"

args = TrainingArguments(
    output_dir = output_dir,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",# no step epoch
    learning_rate = learning_rate,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = 1,
    num_train_epochs = num_train_epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    # _n_gpu=2
)

In [9]:
import numpy as np
def compute_metrics(eval_preds):
    metric = load_metric('glue','mrpc')
    logits,labels = eval_preds # 预测值和真实值
    predictions = np.argmax(logits,axis=-1)
    return metric.compute(predictions=predictions,references=labels)

In [10]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_encoded_dataset,
    eval_dataset=val_encoded_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

### 20230130

In [13]:
trainer.train()

***** Running training *****
  Num examples = 9600
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 24000
/home/anaconda/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.354300,0.357471,0.915833,0.915622
2,0.221000,0.303927,0.944167,0.942784
3,0.130800,0.404569,0.940000,0.939698
4,0.061000,0.457779,0.940000,0.938356
5,0.037100,0.527951,0.935000,0.932642
6,0.037300,0.539391,0.935000,0.932642
7,0.033000,0.541908,0.940833,0.939471
8,0.007500,0.491735,0.942500,0.941376
9,0.022400,0.530432,0.945000,0.943782
10,0.005100,0.549182,0.946667,0.945392


***** Running Evaluation *****
  Num examples = 1200
  Batch size = 2
/home/anaconda/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 1200
  Batch size = 2
/home/anaconda/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 1200
  Batch size = 2
Using the latest cached version of the module from /home/chenli/.cache/huggingface/modules/datasets_modules/metrics/glue/91f3cfc5498873918ecf119dbf806fb10815786c84f41b85a5d3c47c1519b343 (las

TrainOutput(global_step=24000, training_loss=0.09310170924663544, metrics={'train_runtime': 5628.9099, 'train_samples_per_second': 17.055, 'train_steps_per_second': 4.264, 'total_flos': 9581339136458160.0, 'train_loss': 0.09310170924663544, 'epoch': 10.0})

#### 20230206 BERT
旅游评论数据

In [19]:
trainer.train()

***** Running training *****
  Num examples = 33858
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 84650


RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/anaconda/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/home/anaconda/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/anaconda/anaconda3/envs/pytorch/lib/python3.7/site-packages/transformers/models/bert/modeling_bert.py", line 1538, in forward
    return_dict=return_dict,
  File "/home/anaconda/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/anaconda/anaconda3/envs/pytorch/lib/python3.7/site-packages/transformers/models/bert/modeling_bert.py", line 993, in forward
    past_key_values_length=past_key_values_length,
  File "/home/anaconda/anac

In [11]:
trainer.train()

***** Running training *****
  Num examples = 33858
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 84650
/home/anaconda/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.138200,0.224123,0.959357,0.972956
2,0.129100,0.146613,0.968336,0.978567
3,0.098800,0.159872,0.969045,0.979043
4,0.058600,0.151883,0.965737,0.977024
5,0.047200,0.216845,0.962665,0.974992
6,0.029800,0.242301,0.966446,0.977467
7,0.021000,0.231555,0.968100,0.978541
8,0.014200,0.258827,0.966919,0.977714
9,0.011800,0.312368,0.965737,0.976995
10,0.010500,0.340586,0.966919,0.977679


***** Running Evaluation *****
  Num examples = 4232
  Batch size = 2
Saving model checkpoint to /home/chenli/pre_model/BERT_and_Longformer/BERT/checkpoint-8465
Configuration saved in /home/chenli/pre_model/BERT_and_Longformer/BERT/checkpoint-8465/config.json
Model weights saved in /home/chenli/pre_model/BERT_and_Longformer/BERT/checkpoint-8465/pytorch_model.bin
tokenizer config file saved in /home/chenli/pre_model/BERT_and_Longformer/BERT/checkpoint-8465/tokenizer_config.json
Special tokens file saved in /home/chenli/pre_model/BERT_and_Longformer/BERT/checkpoint-8465/special_tokens_map.json
/home/anaconda/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 4232
  Batch size = 2
Using the latest cached version

Saving model checkpoint to /home/chenli/pre_model/BERT_and_Longformer/BERT/checkpoint-76185
Configuration saved in /home/chenli/pre_model/BERT_and_Longformer/BERT/checkpoint-76185/config.json
Model weights saved in /home/chenli/pre_model/BERT_and_Longformer/BERT/checkpoint-76185/pytorch_model.bin
tokenizer config file saved in /home/chenli/pre_model/BERT_and_Longformer/BERT/checkpoint-76185/tokenizer_config.json
Special tokens file saved in /home/chenli/pre_model/BERT_and_Longformer/BERT/checkpoint-76185/special_tokens_map.json
/home/anaconda/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 4232
  Batch size = 2
Saving model checkpoint to /home/chenli/pre_model/BERT_and_Longformer/BERT/checkpoint-84650
Conf

TrainOutput(global_step=84650, training_loss=0.05687479620985357, metrics={'train_runtime': 31228.4134, 'train_samples_per_second': 10.842, 'train_steps_per_second': 2.711, 'total_flos': 1.739924631324e+16, 'train_loss': 0.05687479620985357, 'epoch': 10.0})

评估测试

In [12]:
trainer.evaluate(eval_dataset=test_encoded_dataset)

***** Running Evaluation *****
  Num examples = 4233
  Batch size = 2
/home/anaconda/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Using the latest cached version of the module from /home/chenli/.cache/huggingface/modules/datasets_modules/metrics/glue/91f3cfc5498873918ecf119dbf806fb10815786c84f41b85a5d3c47c1519b343 (last modified on Fri Nov 11 21:04:53 2022) since it couldn't be found locally at glue, or remotely on the Hugging Face Hub.


{'eval_loss': 0.19052943587303162,
 'eval_accuracy': 0.9659815733522324,
 'eval_f1': 0.977639751552795,
 'eval_runtime': 389.8813,
 'eval_samples_per_second': 10.857,
 'eval_steps_per_second': 5.43,
 'epoch': 10.0}